In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(0)

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
class ResNet(nn.Module):
    def __init__(self,dim, m,o):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        
        self.outlayer = nn.Linear(m, o)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y**deg)
        y = self.fc2(y)
        y = F.relu(y**deg)
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y**deg)
        y = self.fc4(y)
        y = F.relu(y**deg)
        y = y+s
        
        
        output = self.outlayer(y)
        return output
deg = 2

In [4]:
def sol_exact(x):
    x_tem = torch.cos(pi*x[:,1:dim_set])
    sol = torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return sol * x[:,0:1]*(1-x[:,0:1])

In [5]:
def sol_exact_H1(x):
    sol = torch.zeros(x.size()[0],3).cuda()
    sol[:,0:1] =  x[:,0:1]*(1-x[:,0:1])*torch.sum(torch.cos(pi*x[:,1:dim_set]),dim=1).reshape([x.size()[0],1])
    sol[:,1:2] = (1-2*x[:,0:1])*torch.sum(torch.cos(pi*x[:,1:dim_set]),dim=1).reshape([x.size()[0],1])
    sol[:,2:dim_set+1] = -pi*x[:,0:1]*(1-x[:,0:1])*torch.sin(pi*x[:,1:dim_set])
    return sol

In [6]:
def right(x):
    x_tem = torch.cos(pi*x[:,1:dim_set])
    f_tem = (-2-pi**2*x[:,0:1]*(1-x[:,0:1]))*torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return f_tem

In [7]:
def BCDataGen(BatchSize):
    bound = torch.rand(BatchSize,1)
    perbd = int(BatchSize/2)
    bound[0:perbd,0]=1.
    bound[perbd:2*perbd,0]=0.
    return bound

In [8]:
dim_set = 2
Batch_size = 10000
def DRM1():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    ux = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    loss1 = torch.sum((ux-du)**2)/Batch_size
    return loss1

In [9]:
def DRM2():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model_u(x)
    du = model_p(x)
    v= torch.ones(u.shape).cuda()
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss2 = torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1])  - right(x))**2)/Batch_size
    return loss2

In [10]:
def model_p(x):
    multi = x*(1-x)
    multi[:,0] = 1
    return model_p1(x)*multi

In [11]:
def model_u(x):
    return (1-x[:,0:1])*x[:,0:1]*model_u1(x) 

In [12]:
model_u1 = ResNet(dim_set,5,1)
model_p1 = ResNet(dim_set,5,dim_set)

device=torch.device("cuda:0" )
model_u1.to(device)
model_p1.to(device)

ResNet(
  (fc1): Linear(in_features=2, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=5, bias=True)
  (fc4): Linear(in_features=5, out_features=5, bias=True)
  (outlayer): Linear(in_features=5, out_features=2, bias=True)
)

In [13]:
notes = torch.rand(10000,dim_set).cuda()
exact = sol_exact_H1(notes)
u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
Du_L2 = sqrt(torch.sum(exact[:,1:3]**2)/10000)
def loss_error():
    notes = torch.rand(10000,dim_set).cuda()
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model_u(notes).reshape([1,notes.size()[0]])
    predict[:,1:dim_set+1]= model_p(notes)
    exact = sol_exact_H1(notes)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    value2 = sqrt(torch.sum((predict[:,1:3] - exact[:,1:3])**2)/10000)/Du_L2
    return value1,value2

In [14]:
traintime = 50000
error_save=np.zeros([2,traintime])
optimizer = optim.Adam([
                {'params': model_u1.parameters()},
                {'params': model_p1.parameters()},
            ])
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses1 = DRM1()
    losses1.backward()
    optimizer.step() 
    optimizer.zero_grad()
    losses2 = DRM2()
    losses2.backward()
    optimizer.step() 
    error1,error2 = loss_error()
    error_save[0,i]=float(error1)
    error_save[1,i]=float(error2)
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses1.detach())
        print("loss2 =",losses2.detach())
        np.save("error_save_LDG_NBC_2D_v2.npy", error_save)
np.save("error_save_LDG_NBC_2D_v2.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 0.8429456037700288
loss1 = tensor(0.0359, device='cuda:0')
loss2 = tensor(7.8512, device='cuda:0')
i=  501
error = 0.012971746076383694
loss1 = tensor(0.0263, device='cuda:0')
loss2 = tensor(0.0168, device='cuda:0')
i=  1001
error = 0.007350485747484159
loss1 = tensor(0.0188, device='cuda:0')
loss2 = tensor(0.0028, device='cuda:0')
i=  1501
error = 0.004104982054124289
loss1 = tensor(0.0150, device='cuda:0')
loss2 = tensor(0.0013, device='cuda:0')
i=  2001
error = 0.004410881482440915
loss1 = tensor(0.0096, device='cuda:0')
loss2 = tensor(0.0011, device='cuda:0')
i=  2501
error = 0.005639773405616111
loss1 = tensor(0.0041, device='cuda:0')
loss2 = tensor(0.0009, device='cuda:0')
i=  3001
error = 0.006682144692697631
loss1 = tensor(0.0031, device='cuda:0')
loss2 = tensor(0.0008, device='cuda:0')
i=  3501
error = 0.003563665694177356
loss1 = tensor(0.0029, device='cuda:0')
loss2 = tensor(0.0003, device='cuda:0')
i=  4001
error = 0.0029419790918125254
loss1 = tensor(0.0027, 

i=  34501
error = 0.0032358869045354052
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(7.4816e-05, device='cuda:0')
i=  35001
error = 0.0015472946421628774
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0002, device='cuda:0')
i=  35501
error = 0.0013556039461196552
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0001, device='cuda:0')
i=  36001
error = 0.0010409357840665703
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0001, device='cuda:0')
i=  36501
error = 0.0007314926357955455
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(6.0256e-05, device='cuda:0')
i=  37001
error = 0.0019740175695569253
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0001, device='cuda:0')
i=  37501
error = 0.001306652552672234
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0001, device='cuda:0')
i=  38001
error = 0.0008374251450659165
loss1 = tensor(0.0002, device='cuda:0')
loss2 = tensor(0.0005, device='cuda:0')
i=  38501
error = 0.0018941088221

In [15]:
import matplotlib.pyplot as plt

In [16]:
index = np.arange(0,50000,10)
plt.plot(np.log10(error_save[index]))

IndexError: index 10 is out of bounds for axis 0 with size 2

In [ ]:
def model_u(x):
    return sol_exact_H1(x)[:,0:1]
def model_p(x):
    return sol_exact_H1(x)[:,1:dim_set+1]